In [1]:
%cd drive

/content/drive


In [3]:
%cd MyDrive/MaskRCNN

/content/drive/MyDrive/MaskRCNN


In [9]:
!python /input/mask_rcnn_images.py --input ../input/image1.jpg

python3: can't open file '/input/mask_rcnn_images.py': [Errno 2] No such file or directory


In [4]:
!pip install opencv-python

  Using cached opencv_python-4.7.0.72-cp37-abi3-win_amd64.whl (38.2 MB)



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
coco_names = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

In [10]:
import cv2
import numpy as np
import random
import torch
import torch
import torchvision
import argparse
from PIL import Image
from torchvision.transforms import transforms as transforms

In [22]:
class MODEL:
    
    def __init__(self):
        
        coco_names = [
        '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
        'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
        'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
        'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
        'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
        'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
        'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
        'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
        'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
        'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
        'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
        'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
        ]
        self.COLORS = np.random.uniform(0, 255, size=(len(coco_names), 3))
        
    def draw_segmentation_map(self,image, masks, boxes, labels):
        alpha = 1 
        beta = 0.6 # transparency for the segmentation map
        gamma = 0 # scalar added to each sum
        for i in range(len(masks)):
            red_map = np.zeros_like(masks[i]).astype(np.uint8)
            green_map = np.zeros_like(masks[i]).astype(np.uint8)
            blue_map = np.zeros_like(masks[i]).astype(np.uint8)
            # apply a randon color mask to each object
            color = self.COLORS[random.randrange(0, len(self.COLORS))]
            red_map[masks[i] == 1], green_map[masks[i] == 1], blue_map[masks[i] == 1]  = color
            # combine all the masks into a single image
            segmentation_map = np.stack([red_map, green_map, blue_map], axis=2)
            #convert the original PIL image into NumPy format
            image = np.array(image)
            # convert from RGN to OpenCV BGR format
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            # apply mask on the image
            cv2.addWeighted(image, alpha, segmentation_map, beta, gamma, image)
            # draw the bounding boxes around the objects
            cv2.rectangle(image, boxes[i][0], boxes[i][1], color=color, 
                          thickness=2)
            # put the label text above the objects
            cv2.putText(image , labels[i], (boxes[i][0][0], boxes[i][0][1]-10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color, 
                        thickness=2, lineType=cv2.LINE_AA)

        return image

    def get_outputs(self,image, model, threshold):
        with torch.no_grad():
            # forward pass of the image through the modle
            outputs = model(image)

        # get all the scores
        scores = list(outputs[0]['scores'].detach().cpu().numpy())
        # index of those scores which are above a certain threshold
        thresholded_preds_inidices = [scores.index(i) for i in scores if i > threshold]
        thresholded_preds_count = len(thresholded_preds_inidices)
        # get the masks
        masks = (outputs[0]['masks']>0.5).squeeze().detach().cpu().numpy()
        # discard masks for objects which are below threshold
        masks = masks[:thresholded_preds_count]
        # get the bounding boxes, in (x1, y1), (x2, y2) format
        boxes = [[(int(i[0]), int(i[1])), (int(i[2]), int(i[3]))]  for i in outputs[0]['boxes'].detach().cpu()]
        # discard bounding boxes below threshold value
        boxes = boxes[:thresholded_preds_count]
        # get the classes labels
        labels = [coco_names[i] for i in outputs[0]['labels']]
        return masks, boxes, labels

    
    

In [17]:
### Intializing Mask RCNN and generating output

In [18]:
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True, progress=True, 
                                                           num_classes=91)
# set the computation device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# load the modle on to the computation device and set to eval mode
model.to(device).eval()

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [29]:
# import the modules

import os
from os import listdir

# get the path/directory
folder_dir = "Dataset/images/test"
for images in os.listdir(folder_dir):
# check if the image ends with png
    if (images.endswith(".jpg")):
        print(images)


0671a3ea-Test_Img_10.jpg
130d0add-Test_Img_8.jpg
61ece895-Test_Img_9.jpg
62e9c732-Test_Img_1.jpg
7554dbb7-Test_Img_4.jpg
7ca25b6e-Test_Img_2.jpg
8b46a742-Test_Img_3.jpg
9b4dc52b-Test_Img_12.jpg
a050c61d-Test_Img_7.jpg
c660de6b-Test_Img_11.jpg
d18965a8-Test_Img_13.jpg
d1a325af-Test_Img_5.jpg
f248a791-Test_Img_6.jpg


In [40]:

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((1080,1080)),
    transforms.Normalize((0.0,0.0,0.0),(255.,255.,255.))
])

import os
from os import listdir

# get the path/directory
folder_dir = "Dataset/images/test"
for images in os.listdir(folder_dir):
    # from google.colab.patches import cv2_imshow
    image = Image.open(f"Dataset/images/test/{images}").convert('RGB')
    # keep a copy of the original image for OpenCV functions and applying masks
    orig_image = image.copy()
    # transform the image
    image = transform(image)
    # add a batch dimension
    print(type(image))
    image = image.unsqueeze(0)
    segmentation = MODEL()
    masks, boxes, labels = segmentation.get_outputs(image, model, 0.9)
    result = segmentation.draw_segmentation_map(orig_image, masks, boxes, labels)
    print(result)
    # visualize the image
#     cv2.imshow('Segmented image', result)
#     cv2.waitKey(0)
    save_path = f"../MASKRCNN_outputs/{images}"
    cv2.imwrite(save_path, result)

<class 'torch.Tensor'>
<PIL.Image.Image image mode=RGB size=4032x3024 at 0x18D83CF0760>


error: OpenCV(4.7.0) :-1: error: (-5:Bad argument) in function 'imwrite'
> Overload resolution failed:
>  - img is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'img'


In [93]:
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, img_paths):
        self.img_paths = img_paths
        self.transform = transforms.Compose([torchvision.transforms.Resize((1080,1080)),
                                                         torchvision.transforms.Normalize((0.0,0.0,0.0),(255.,255.,255.))])

    def __getitem__(self, index):
        # Load image
        img_path = self.img_paths[index]
        img = Image.open(img_path).convert('RGB')
        
        # Apply transforms and convert to tensor
        transformed_img = self.transform(img)
        tensor_img = torch.Tensor(transformed_img)
        
        return (tensor_img, img)

    def __len__(self):
        return len(self.img_paths)


In [94]:
d = (CustomDataset('Dataset/images/train'),CustomDataset('Dataset/images/val'),CustomDataset('Dataset/images/test'))
   

In [95]:
def dataset_evaluation():
    
    for i in range(0,3):
        
        dt_set = d[i]
        train_dataloader = torch.utils.data.DataLoader(dt_set,batch_size=9)
        print(train_dataloader)
        for batch in train_dataloader:            
            mages,annotated_imgs = batch        
            segmentation = MODEL()
            masks, boxes, labels = segmentation.get_outputs(image, model, 0.9)
            result = segmentation.draw_segmentation_map(orig_image, masks, boxes, labels)
            print(result)

            save_path = f"../MASKRCNN_outputs/{images}"
            cv2.imwrite(save_path, result)




In [96]:
dataset_evaluation()

FileNotFoundError: [Errno 2] No such file or directory: 'D'

### TRAINING THE DATASET WITH MASK RCNN

In [99]:
import torch
from torchvision.models.detection import maskrcnn_resnet50_fpn
from my_dataset import MyDataset  # Custom dataset class
from torch.utils.data import DataLoader
from torch.optim import SGD
import torch.nn.functional as F

# Define your model
model = maskrcnn_resnet50_fpn(num_classes=3)

# Prepare your data loaders
train_dataset = MyDataset('Dataset/images/train')  # Replace 'train' with the name of your training dataset
val_dataset = MyDataset('Dataset/images/val')  # Replace 'val' with the name of your validation dataset
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=4)

# Define your optimizer and loss function
optimizer = SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)
loss_fn = F.cross_entropy

# Train your model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, targets in train_loader:
        optimizer.zero_grad()
        loss = 0
        for i in range(len(images)):
            output = model(images[i].unsqueeze(0))
            loss += loss_fn(output['masks'][0], targets[i]['masks'])
        loss.backward()
        optimizer.step()
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for images, targets in val_loader:
            for i in range(len(images)):
                output = model(images[i].unsqueeze(0))
                val_loss += loss_fn(output['masks'][0], targets[i]['masks'])
    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}')

#


ModuleNotFoundError: No module named 'my_dataset'